In [29]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
from dash.dependencies import Input, Output

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.json_util import dumps
from pymongo import MongoClient
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
table_data = shelter.read({}, { "_id":0})
df = pd.DataFrame.from_records(table_data)

logo = 'logo.png'

def b64_image(image_filename):
    with open(image_filename, 'rb') as f:
        image = f.read()
    return 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div',children=[
            html.H6("Developed by Austin Layman"),
              ]),
    html.Div(className="row",
            style={"display" : "flex", 
                   "column-count" : "auto", 
                   "column-gap" : "8px", 
                   "align-items" : "center", 
                   "justify-content" : "center"},
        
                children=[
                    html.Img(src=b64_image(logo),
                            height="225px",
                            width="225px"),
                    html.B(html.H1('Animal Training Database')),
                ],
    ),
    
    html.Hr(),
    html.Div(className="row",
             style={"display" : "flex", "column-count" : "auto", "column-gap" : "8px", "line-height" : "normal"},
             children=[

                 dcc.Dropdown(

                     # Get dropdown options from unique values in dataset
                    options=[
                        {"label" : "Water Rescue", "value" : "water"},
                        {"label" : "Mountain/Wilderness Rescue", "value" : "mtn_wild"},
                        {"label" : "Disaster Rescue or Individual Tracking", "value" : "disaster"}
                    ],

                    id="rescue-dropdown",
                    placeholder="Training Type",
                    style={"width": "300px"}
                ),

      
                
         ]),
    html.Br(),

    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10, 
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,

    ),
    
    html.Br(),
    
    html.Hr(),

    
    html.Div(className='row',
         style={"display" : "flex", "column-count" : "auto", "column-gap" : "8px", "line-height" : "normal"},
             children=[
                    dcc.Graph(id="graph-id"),
                 
                    dl.Map(id="map-id", 
                           style={'width': '1000px', 'height': '500px'}, 
                           center=[30.75,-97.48], 
                           zoom=10, 
                           children=[
                                dl.TileLayer(id="base-layer-id"),
                                dl.MarkerClusterGroup(id="map-markers"),
                           ])
                    ]
            )
         
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)


def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback updates the data table values when inputs are changed
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-dropdown', 'value')]
)

def dropdown_filter(rescue_value):
    
    
    # Query if water rescue is selected from dropdown    
    if rescue_value == "water":
        query = shelter.read({"animal_type":"Dog",
                                   "breed":{"$in":["Labrador Retriever Mix",
                                                   "Chesapeake Bay Retriever",
                                                   "Newfoundland"]},
                                   "sex_upon_outcome":"Intact Female",
                                   "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}},
                                  { "_id":0})
    
    # Query if mountain/wilderness rescue is selected from dropdown
    elif rescue_value == "mtn_wild":
        query = shelter.read({"animal_type":"Dog",
                                   "breed":{"$in":["German Shepherd",
                                                   "Alaskan Malamute",
                                                   "Old English Sheepdog",
                                                   "Siberian Husky",
                                                   "Rottweiler"]},
                                   "sex_upon_outcome":"Intact Male",
                                   "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}},
                                  { "_id":0})
        
    # Query if disaster rescue is selected from dropdown
    elif rescue_value == "disaster":
        query = shelter.read({"animal_type":"Dog",
                                   "breed":{"$in":["Doberman Pinscher",
                                                   "German Shepherd",
                                                   "Golden Retriever",
                                                   "Bloodhound",
                                                   "Rottweiler"]},
                                   "sex_upon_outcome":"Intact Male",
                                   "age_upon_outcome_in_weeks":{"$gte":20, "$lte":300}},
                                  { "_id":0})
        
    
    # Query if no option is selected from dropdown
    else:
        query = shelter.read({}, { "_id":0})
        
        
    dff = pd.DataFrame.from_records(query)    

    return dff.to_dict('records')


# This callback updated the geolocation data
@app.callback(
    Output('map-markers', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_map(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    row_count = dff.shape[0]
    
    # Limit number of markers to 1000
    if row_count > 1000:
         row_count = 1000
    
    # Create a list of markers
    markers = []
    for i in range(row_count):
         markers.append(
             dl.Marker(position=[dff.iloc[i,13],dff.iloc[i,14]], children=[
                        dl.Tooltip(dff.iloc[i,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[i,9])
                        ])
                    ])
             )
         
         
    
    return markers

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(data_frame = dff, names=dff["breed"], hole = 0.0)
    fig.update_traces(textinfo='value')
    return fig


app